# Creating classifier and vectorizer for the model

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords

from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [2]:
if Version(sklearn_version) < '0.18':
    clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
else:
    clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

In [3]:
def stream_docs(path):
  with open(path, 'r', encoding='utf-8') as csv:
      next(csv)  # skip header
      for line in csv:
          text, label = line[:-3], int(line[-2])
          yield text, label

In [4]:
def get_minibatch(doc_stream, size):
  docs, y = [], []
  try:
      for _ in range(size):
          text, label = next(doc_stream)
          docs.append(text)
          y.append(label)
  except StopIteration:
      return None, None
  return docs, y

In [5]:
def tokenizer(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
  tokenized = [w for w in text.split() if w not in stop]
  return tokenized

In [6]:
doc_stream = stream_docs(path='movie_data.csv')
stop = stopwords.words('english')
stop.remove('not')

In [7]:
vect = HashingVectorizer(decode_error='ignore', 
                       n_features=2**21,
                       preprocessor=None, 
                       tokenizer=tokenizer)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

In [8]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.869


## stopwords

'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", its','itself',
'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 
'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in',
'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 
'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 
'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 
'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
"hadn't",'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', 
"needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',"won't", 'wouldn', "wouldn't"

# pickle creator for storing model

In [9]:
import pickle
import os

In [10]:
dest = os.path.join('movie_classifier_output')

In [11]:
if not os.path.exists(dest):
  os.makedirs(dest)

In [12]:
pickle.dump(stop,open(os.path.join(dest,'stopwords.pkl'),'wb'),protocol=4)

In [13]:
pickle.dump(clf, open(os.path.join(dest,'classifier.pkl'),'wb'),protocol=4)